# export tiles for gain and loss
Based on [ee code](https://code.earthengine.google.com/8423c718d647f15ab3e9c31868489fd4?noload=true)

In [26]:
import ee
import geemap

In [2]:
# Trigger the authentication flow.
#ee.ServiceAccountCredentials(EE_ACCOUNT, EE_PRIVATE_KEY_FILE)
ee.Authenticate()
# Initialize the library.
ee.Initialize()


Successfully saved authorization token.


### Data Assets

In [32]:
gcs_bucket = 'mangrove_atlas'
dataset = 'mangrove_blue_carbon'

In [1]:
## Assets selection
# region for export
region = ee.Geometry.Polygon([[[-180, 33],[-180, -34],[180, -34],[180, 33]]], None, False);
# gain
gain = ee.ImageCollection(
    "projects/global-mangrove-watch/land-cover/mangrove-extent-gain")
# loss
loss = ee.ImageCollection(
    "projects/global-mangrove-watch/land-cover/mangrove-extent-loss")

NameError: name 'ee' is not defined

In [ ]:
# styles
gain_ramp ='''
    <RasterSymbolizer>
        <ColorMap type="ramp" extended="false" >
        <ColorMapEntry color="#a6cb10" quantity="1" label="1"/>
        </ColorMap>
    </RasterSymbolizer>
'''
loss_ramp = '''
    <RasterSymbolizer>
        <ColorMap type="ramp" extended="false" >
        <ColorMapEntry color="#eb6240" quantity="1" label="1"/>
        </ColorMap>
    </RasterSymbolizer>
'''

In [161]:
def exportMapTasks(asset: ee.ImageCollection, asset_name: str, gcbucket: str, years: list,
                    region: ee.Geometry, style: str, min_zoom: int, max_zoom: int, env: str = 'staging'):
    """
    Export the data to GEE.

    Parameters
    ----------
    asset : ee.Image
        The image to export.
    asset_name : str
        The name of the asset.
    gcbucket : str
        The name of the GCS bucket.
    geometry_collection : ee.GeometryCollection
        The geometry collection to export the data.
    
    Returns
    -------
    List of tasks
    """
    years = asset.date().getInfo()['year']
    taskList = []
    
    # TODO: improve this loop
    for year in years:
        task = ee.batch.Export.map.toCloudStorage(
            image = ee.Image(asset.filterMetadata('year', 'equals', year)
                    ).sldStyle(style),
            description = f'{asset_name}_{year}',
            path = f'{env}/tilesets/{asset_name}/{year}',
            bucket = gcbucket,
            minZoom = min_zoom,
            maxZoom = max_zoom,
            writePublicTiles = True,
            skipEmptyTiles = True,
            fileFormat = 'png',
            region = region
            )
        taskList.append(task)
    
    return taskList

In [ ]:
def batchExecute(taskList: list, batch_size: int = 20):
    """
    Execute the tasks in the list in baches of 20 as is the max allowed by GEE. 
    each task takes about 10 min to execute.
    """
    n_tasks = len(taskList)
    n_batches = Math.ceil(n_tasks / batch_size)
    
    for i in range(n_batches):
        for task in taskList[i*batch_size:(i+1)*batch_size]:
            task.start()
        
        for task in taskList[i*batch_size:(i+1)*batch_size]:
            task.wait()

    return taskList

### Vizualize the data prior exporting it

In [201]:
Map = geemap.Map(center=(-5, 39), zoom=12, basemap='HYBRID')
vis_params = {
    'min': 1,
    'max': 1,
    'palette': ['eb6240'],
}
Map.addLayer(gain.first(), vis_params, 'Gain', True, 0.5)
Map.addLayer(loss.first(), vis_params, 'Loss', True, 0.5)
Map

Map(center=[-5, 39], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [ ]:
# Create a list of tasks to export the map tiles
gain_export_tasks = exportMapTasks(gain, 'gain', gcs_bucket, gain.date().getInfo()['year'], 
            region, gain_ramp, 0, 12)

In [ ]:
# Create a list of tasks to export the map tiles
loss_export_tasks = exportMapTasks(gain, 'gain', gcs_bucket, gain.date().getInfo()['year'], 
            region, gain_ramp, 0, 12)

In [ ]:
tasks_list = gain_export_tasks + loss_export_tasks

In [ ]:
batchExecute(tasks_list)